## <center> Online Retail Preprocessing </center>
---

**Name**: Francisco Aarón Ortega Anguiano

This practice exercise involves preprocessing and exploratory data analysis (EDA) on a dataset sourced from **Kaggle**.

### Dataset columns:
- `InvoiceNo`
- `InvoiceDate`
- `InvoiceTime`
- `StockCode`
- `Description`
- `Quantity`
- `UnitPrice`
- `Totalsale`
- `CustomerID`
- `Country`

#### Import important libraries and read the dataset

In [1]:
import pandas as pd

retail_df = pd.read_csv('db/Online-Retail.csv')

retail_df.head()

,InvoiceNo,InvoiceDate,InvoiceTime,StockCode,Description,Quantity,UnitPrice,Totalsale,CustomerID,Country
0,536365,01-12-2010,08:26:00 AM,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2.55,15.30,17850.0,United Kingdom
1,536365,01-12-2010,08:26:00 AM,71053,WHITE METAL LANTERN,6,3.39,20.34,17850.0,United Kingdom
2,536365,01-12-2010,08:26:00 AM,84406B,CREAM CUPID HEARTS COAT HANGER,8,2.75,22.00,17850.0,United Kingdom
3,536365,01-12-2010,08:26:00 AM,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,3.39,20.34,17850.0,United Kingdom
4,536365,01-12-2010,08:26:00 AM,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,3.39,20.34,17850.0,United Kingdom


In [2]:
print(f"data shape: {retail_df.shape}")

data shape: (541909, 10)


#### Check for duplicates
Our data has duplicates that we are going to drop

In [3]:
print(f"Our data has: {retail_df.duplicated().sum()} duplicates")

Our data has: 5268 duplicates


In [4]:
retail_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 10 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   InvoiceNo    541909 non-null  object 
 1   InvoiceDate  541909 non-null  object 
 2   InvoiceTime  541909 non-null  object 
 3   StockCode    541909 non-null  object 
 4   Description  540455 non-null  object 
 5   Quantity     541909 non-null  int64  
 6   UnitPrice    541909 non-null  float64
 7   Totalsale    541909 non-null  float64
 8   CustomerID   406829 non-null  float64
 9   Country      541909 non-null  object 
dtypes: float64(3), int64(1), object(6)
memory usage: 41.3+ MB


#### Check the count of null values per column in the DataFrame
We can drop the missing values in the description, but it has a lot of nulls in CustomerID which isn't very good

In [5]:
retail_df.isnull().sum()

InvoiceNo           0
InvoiceDate         0
InvoiceTime         0
StockCode           0
Description      1454
Quantity            0
UnitPrice           0
Totalsale           0
CustomerID     135080
Country             0
dtype: int64

#### Describe to have a fast overview
We can see that `Quantity`, `UnitPrice` and `Totalsale` have negative values, we are going to explore why.

In [6]:
retail_df.describe()

,Quantity,UnitPrice,Totalsale,CustomerID
count,541909.000000,541909.000000,541909.000000,406829.000000
mean,9.552250,4.611114,17.987795,15287.690570
std,218.081158,96.759853,378.810824,1713.600303
min,-80995.000000,-11062.060000,-168469.600000,12346.000000
25%,1.000000,1.250000,3.400000,13953.000000
50%,3.000000,2.080000,9.750000,15152.000000
75%,10.000000,4.130000,17.400000,16791.000000
max,80995.000000,38970.000000,168469.600000,18287.000000


I'm going to start by dividing the data into two different dataframes one with the non negative values and the other containing them, to see if the values should be negative or are a mistake that needs to be fixed.

In [7]:
no_neg_values_df = retail_df[
	(retail_df['Quantity'] >= 0) &
	(retail_df['UnitPrice'] >= 0) &
	(retail_df['Totalsale'] >= 0)
]

no_neg_values_df.describe()

,Quantity,UnitPrice,Totalsale,CustomerID
count,531283.000000,531283.000000,531283.000000,397924.000000
mean,10.655299,3.898954,20.077218,15294.315171
std,156.830617,35.876279,270.058254,1713.169877
min,1.000000,0.000000,0.000000,12346.000000
25%,1.000000,1.250000,3.750000,13969.000000
50%,3.000000,2.080000,9.900000,15159.000000
75%,10.000000,4.130000,17.700000,16795.000000
max,80995.000000,13541.330000,168469.600000,18287.000000


In [8]:
neg_values_df = retail_df[
	(retail_df['Quantity'] < 0) |
	(retail_df['UnitPrice'] < 0) |
	(retail_df['Totalsale'] < 0)
]

neg_values_df.describe()

,Quantity,UnitPrice,Totalsale,CustomerID
count,10626.000000,10626.000000,10626.000000,8905.000000
mean,-45.598438,40.217975,-86.480012,14991.667266
std,1092.111602,641.765091,1913.347799,1706.772357
min,-80995.000000,-11062.060000,-168469.600000,12346.000000
25%,-10.000000,1.060000,-15.950000,13510.000000
50%,-2.000000,2.100000,-6.800000,14895.000000
75%,-1.000000,4.950000,-2.100000,16393.000000
max,1.000000,38970.000000,0.000000,18282.000000


In [9]:
neg_values_df.sample(10)

,InvoiceNo,InvoiceDate,InvoiceTime,StockCode,Description,Quantity,UnitPrice,Totalsale,CustomerID,Country
448618,C575065,08-11-2011,12:40:00 PM,23227,FILIGREE HEART BUTTERFLY WHITE,-3,1.25,-3.75,13924.0,United Kingdom
363689,C568570,28-09-2011,09:04:00 AM,22558,CLOTHES PEGS RETROSPOT PACK 24,-2,1.65,-3.30,15358.0,United Kingdom
333167,C566180,09-09-2011,12:52:00 PM,21314,SMALL GLASS HEART TRINKET POT,-1,2.10,-2.10,16173.0,United Kingdom
217091,C555882,07-06-2011,03:40:00 PM,22842,BISCUIT TIN VINTAGE RED,-1,6.75,-6.75,14625.0,United Kingdom
148085,549161,06-04-2011,05:15:00 PM,21518,NaN,-31,0.00,0.00,NaN,United Kingdom
362300,C568420,27-09-2011,11:17:00 AM,22720,SET OF 3 CAKE TINS PANTRY DESIGN,-1,4.95,-4.95,16573.0,United Kingdom
230551,C557175,17-06-2011,11:13:00 AM,22962,JAM JAR WITH PINK LID,-3,0.85,-2.55,16170.0,United Kingdom
535329,581206,07-12-2011,06:34:00 PM,21693,mixed up,-87,0.00,0.00,NaN,United Kingdom
342635,C566899,15-09-2011,01:53:00 PM,AMAZONFEE,AMAZON FEE,-1,7427.97,-7427.97,NaN,United Kingdom
369956,C569114,30-09-2011,12:50:00 PM,22832,BROCANTE SHELF WITH HOOKS,-2,10.75,-21.50,14911.0,EIRE


In [10]:
neg_values_df[(neg_values_df['UnitPrice'] < 0)]

,InvoiceNo,InvoiceDate,InvoiceTime,StockCode,Description,Quantity,UnitPrice,Totalsale,CustomerID,Country
299983,A563186,12-08-2011,02:51:00 PM,B,Adjust bad debt,1,-11062.06,-11062.06,NaN,United Kingdom
299984,A563187,12-08-2011,02:52:00 PM,B,Adjust bad debt,1,-11062.06,-11062.06,NaN,United Kingdom


In [11]:
no_neg_values_df.isnull().sum()

InvoiceNo           0
InvoiceDate         0
InvoiceTime         0
StockCode           0
Description       592
Quantity            0
UnitPrice           0
Totalsale           0
CustomerID     133359
Country             0
dtype: int64

In [12]:
neg_values_df.isnull().sum()

InvoiceNo         0
InvoiceDate       0
InvoiceTime       0
StockCode         0
Description     862
Quantity          0
UnitPrice         0
Totalsale         0
CustomerID     1721
Country           0
dtype: int64

In [13]:
print(f"no negative values data shape: {no_neg_values_df.shape}")
print(f"negative values data shape: {neg_values_df.shape}")

no negative values data shape: (531283, 10)
negative values data shape: (10626, 10)


It seems like the columns `Quantity`, `UnitPrice` and `Totalsale` have negative for different motives related to the business, so for now we are going to focus exclusively in the data that has no negatives in it. We can see that most of the null values are from this part of the data, which isn't surprisely seen that most of the data are non negative.

---

#### Clean the data

- Drop duplicates
- Drop all the values in `Description`
- `CustomerID` has almost 140k missing values, almost 25% of the data

In [14]:
# make a copy of the non negative df
retail_df2 = no_neg_values_df.copy()

In [15]:
# dropping all duplicates
retail_df2.drop_duplicates(inplace=True)

# resetting the index
retail_df2.reset_index(drop=True, inplace=True)

In [16]:
# dropping missing values
retail_df2.dropna(subset=['Description'], inplace=True)

# making all items lower
retail_df2['Description'] = retail_df2['Description'].apply(str.lower)

# remove leading and trailing white spaces, and removing "."
retail_df2['Description'] = retail_df2['Description'].apply(lambda x: x.strip().replace(".", ""))

In [17]:
# replace the missing values to 0
retail_df2['CustomerID'] = retail_df2['CustomerID'].fillna(0)

# another solution is to drop the rows with missing CustomerID, however in this case it's less recommended for the kind of analysis that I will be doing since nearly 140k rows would be lost, resulting in a significant loss of information.
# retail_df2.dropna(subset=['CustomerID'], inplace=True)

In [18]:
retail_df2.isnull().sum()

InvoiceNo      0
InvoiceDate    0
InvoiceTime    0
StockCode      0
Description    0
Quantity       0
UnitPrice      0
Totalsale      0
CustomerID     0
Country        0
dtype: int64

#### Simple Feature Engineering

In [19]:
from lib.feature_engineering import FeatureEngineering

fe = FeatureEngineering(retail_df2)

# making sure all the invoice dates are formatted well
fe.convert_date_column()
retail_df2 = fe.data

In [20]:
# extracting Year and Month columns
fe.extract_year_and_month_column()
retail_df2 = fe.data

In [21]:
# Add the season column
fe.add_season_column()
retail_df2 = fe.data

In [22]:
retail_df2[['InvoiceDate', 'Year', 'Month', 'Season']].sample(5)

,InvoiceDate,Year,Month,Season
388340,2011-10-16,2011,10,Fall
432612,2011-11-07,2011,11,Fall
250729,2011-07-11,2011,7,Summer
257948,2011-07-15,2011,7,Summer
369577,2011-10-06,2011,10,Fall


In [23]:
retail_df2['AM_or_PM'] = retail_df2['InvoiceTime'].apply(fe.extract_am_pm)

In [24]:
retail_df2[['InvoiceTime', 'AM_or_PM']].sample(5)

,InvoiceTime,AM_or_PM
495048,12:52:00 PM,PM
480006,01:39:00 PM,PM
254557,01:14:00 PM,PM
122134,01:28:00 PM,PM
81021,11:56:00 AM,AM


In [25]:
retail_df2.info()

<class 'pandas.core.frame.DataFrame'>
Index: 525460 entries, 0 to 526051
Data columns (total 14 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   InvoiceNo    525460 non-null  object        
 1   InvoiceDate  525460 non-null  datetime64[ns]
 2   InvoiceTime  525460 non-null  object        
 3   StockCode    525460 non-null  object        
 4   Description  525460 non-null  object        
 5   Quantity     525460 non-null  int64         
 6   UnitPrice    525460 non-null  float64       
 7   Totalsale    525460 non-null  float64       
 8   CustomerID   525460 non-null  float64       
 9   Country      525460 non-null  object        
 10  Year         525460 non-null  int32         
 11  Month        525460 non-null  int32         
 12  Season       525460 non-null  object        
 13  AM_or_PM     525460 non-null  object        
dtypes: datetime64[ns](1), float64(3), int32(2), int64(1), object(7)
memory usage: 56.1+ MB


In [26]:
# save the new transformed data into a csv
retail_df2.to_csv('db/Online-Retail-Transformed.csv', index=False)

---
### Next part of the project
Once finished with the preprocessing of the data we can start a EDA

[2. Exploratory Data Analysis Univariate](2.online_retail_univariate.ipynb)